In [1]:
import os
import re
from collections import namedtuple

In [2]:
devset_dir = "factRuEval-2016/devset"
testset_dir = "factRuEval-2016/testset"
result_dir = "factRuEval-2016/results"

FactRu = namedtuple('FactRu', ['id', 'text'])

In [3]:
def read_dataset(dir_path, filetype):
    for filename in os.listdir(dir_path):
        match = re.match('book_(\d+)\.'+filetype, filename)
        if match:
            id = int(match.group(1))
            path = os.path.join(dir_path, filename)
            with open(path) as f:
                text = list()
                for line in f:
                    items = line.split()
                    if len(items)>1:
                        if filetype == 'tokens':
                            text.append(items[3])
                        else:
                            entity = list()
                            entity.append(items[1])
                            counter = 0
                            while items[counter+2] != '#':
                                counter += 1
                            ind_begin = counter+3
                            while counter > 0:
                                entity.append(items[ind_begin])
                                counter -= 1
                                ind_begin += 1
                            text.append(entity)
            yield FactRu(id, text)

In [4]:
train_tokens = list(read_dataset(devset_dir, 'tokens'))
test_tokens = list(read_dataset(testset_dir, 'tokens'))
train_objects = list(read_dataset(devset_dir, 'objects'))
test_objects = list(read_dataset(testset_dir, 'objects'))

In [5]:
print(train_tokens[0].text)

['Правительство', 'Японии', 'выразило', 'решительный', 'протест', 'против', 'состоявшегося', 'в', 'пятницу', 'визита', 'российского', 'министра', 'обороны', ',', 'посетившего', 'южные', 'Курильские', 'острова', ',', 'которые', 'и', 'Япония', ',', 'и', 'Россия', 'считают', 'своими', '.', 'Четыре', 'острова', ',', 'которые', 'называются', 'Южными', 'Курилами', ',', 'и', 'которые', 'японцы', 'называют', 'Северными', 'территориями', ',', 'отошли', 'к', 'Советскому', 'Союзу', 'после', 'Второй', 'мировой', 'войны', '.', 'Территориальный', 'спор', 'вокруг', 'этих', 'островов', 'до', 'сих', 'пор', ',', 'через', '65', 'лет', 'после', 'окончания', 'военных', 'действий', ',', 'не', 'дает', 'двум', 'странам', 'заключить', 'мирное', 'соглашение', '.', 'Премьер-министр', 'Японии', 'Наото', 'Кан', 'назвал', 'посещение', 'двух', 'из', 'этих', 'островов', 'министром', 'обороны', 'России', 'Анатолием', 'Сердюковым', '«', 'в', 'высшей', 'степени', 'достойным', 'сожаления', '»', '.', 'Эта', 'поездка', 'со

In [6]:
print(train_objects[0].text)

[['LocOrg', 'США'], ['Org', 'комитетом', 'ООН', 'комитетом'], ['Org', 'ООН'], ['Location', 'Женеве'], ['Org', 'министерства', 'иностранных', 'дел'], ['Person', 'Барри', 'Лоуенкрон'], ['Person', 'Barry', 'Lowenkron'], ['LocOrg', 'США'], ['Org', 'США', 'правительства'], ['Person', 'Лоуенкрон'], ['LocOrg', 'США'], ['Org', 'США', 'правительство'], ['Location', 'Абу-Грэйв'], ['Org', 'Абу-Грэйв', 'тюрьме'], ['Org', 'ЦРУ'], ['Org', 'Аль-Каиды'], ['LocOrg', 'США'], ['Org', 'США', 'делегация'], ['Org', 'Международная'], ['Org', 'Amnesty'], ['LocOrg', 'США'], ['Org', 'лагере', 'Гуантанамо'], ['Location', 'Гуантанамо'], ['Location', 'Кубе'], ['Location', 'Ираке'], ['Location', 'Афганистане'], ['LocOrg', 'США'], ['Org', 'ЦРУ'], ['Location', 'Македонии'], ['LocOrg', 'Германии'], ['Person', 'Халеда', 'эль-Масри'], ['Person', 'Khaled', 'el-Masri'], ['Location', 'Афганистан'], ['Org', 'Аль-Каиды'], ['Location', 'Сирию']]


In [7]:
from ufal.udpipe import Model, Pipeline

def tag_ud(text='Текст нужно передать функции в виде строки!', modelfile='udpipe_syntagrus.model'):
    model = Model.load(modelfile)
    pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
    processed = pipeline.process(text) # обрабатываем текст, получаем результат в формате conllu
    output = [l for l in processed.split('\n') if not l.startswith('#')] # пропускаем строки со служебной информацией
    tagged = [w.split('\t')[2].lower() + '_' + w.split('\t')[3] for w in output if w] # извлекаем из обработанного текста лемму и тэг
#     tagged_propn = []
#     propn  = []
#     print(tagged)
#     for t in tagged:
#         if t.endswith('PROPN'):
#             if propn:
#                 propn.append(t)
#             else:
#                 propn = [t]
#         else:
#             if len(propn) > 1:
#                 for x in propn:
#                     #name = '::'.join([x.split('_')[0] for x in propn]) + '_PROPN'
#                     tagged_propn.append(x)
#             elif len(propn) == 1:
#                 tagged_propn.append(propn[0])
#             tagged_propn.append(t)
#             propn = []
    return tagged

In [8]:
def make_xy(tokens, objects):
    xy_list = list()
    tokens_list = list()
    tokens_str = ''
    tags_list = list()
    new_tokens_list = list()
    for i in range(len(tokens)):
        print(i)
        tokens_list_for_id = tokens[i]
        tags_list_for_id = [tags for tags in objects if tokens_list_for_id.id == tags.id]
        tokens_list_for_id_text = tokens_list_for_id.text
        tags_list_for_id_text = tags_list_for_id[0].text
        prev_tag = None
        tag = None
        for ind in range(len(tokens_list_for_id_text)):
            token = tokens_list_for_id_text[ind]
            tokens_str += token + ' '
            for tags in tags_list_for_id_text:
                match = [word for word in tags if word == token]
                if len(match) > 0:
                    tag = tags[0]
            if tag == None:
                tag = 'O'
            elif tag == 'Person':
                tag = 'PER'
            elif tag == 'Org':
                tag = 'ORG'
            elif tag == 'Location' or tag =='LocOrg':
                tag = 'LOC'
            else:
                tag = 'MISC'			
            if tag == prev_tag and tag != 'O':
                prev_tag = tag
                tag = 'I-' + tag
            elif tag != 'O':
                prev_tag = tag
                tag = 'B-' + tag
            else:
                prev_tag = tag
            tokens_list.append(token)
            tags_list.append(tag)
            tag = None
            if token == '.':
                new_tokens_list = tag_ud(tokens_str, 'rus_emb/'+'udpipe_syntagrus.model')
                #print((new_tokens_list, tags_list, ))
                print(len(tokens_list), len(new_tokens_list))
                xy_list.append((new_tokens_list, tags_list,))
                tokens_list = list()
                new_tokens_list = list()
                tags_list = list()
                tokens_str = ''
    return xy_list

In [ ]:
xy_list = make_xy(train_tokens, train_objects)

0
28 28
24 24
25 25
23 23
18 18
1
29 29
45 45
40 40
32 32
2
18 18
24 24
16 16
25 25
7 7
17 17
9 9
12 12
15 15
23 23
31 31
17 17
31 31
11 11
19 19
3
19 19
18 18
28 28
14 14
25 25
22 22
13 13
4 4
4
16 16
35 35
5
40 41
15 15
12 12
44 44
15 15
11 11
18 18
16 16
6
3 3
16 16
33 33
21 21
21 21
24 24
37 37
12 12
7
31 31
20 20
28 28
23 23
36 36
22 22
2 2
24 24
22 22
29 29
17 17


KeyboardInterrupt: 

In [9]:
tag_ud("23:30 подстригли дата-центры", 'rus_emb/'+'udpipe_syntagrus.model')

['23_NUM', ':_PUNCT', '30_NUM', 'подстригль_VERB', 'дата_NOUN', '-центр_NOUN']

In [14]:
re.findall(r"[\w']+|[.,!?;]", "23:30 подстригли дата-центры")

['23', '30', 'подстригли', 'дата', 'центры']

In [17]:
import gensim
embeddings_filepath='rus_emb/ruwikiruscorpora-nobigrams_upos_skipgram_300_5_2018.vec.gz'

In [18]:
model = gensim.models.KeyedVectors.load_word2vec_format(embeddings_filepath, binary=False)

In [ ]:
'что_SCONJ' in model.vocab